In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
import torchvision.models as models

In [2]:
reference = models.resnet50(pretrained=True)

In [3]:
reference

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [4]:
reference.state_dict().keys()

odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.0.conv3.weight', 'layer1.0.bn3.weight', 'layer1.0.bn3.bias', 'layer1.0.bn3.running_mean', 'layer1.0.bn3.running_var', 'layer1.0.bn3.num_batches_tracked', 'layer1.0.downsample.0.weight', 'layer1.0.downsample.1.weight', 'layer1.0.downsample.1.bias', 'layer1.0.downsample.1.running_mean', 'layer1.0.downsample.1.running_var', 'layer1.0.downsample.1.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.we

In [85]:
class ResNet(nn.Module):
    def __init__(self, **params):
        super().__init__()
        
        self.dim = params['dim']
        self.expansion = params['expansion']
        self.num_classes = params['num_classes']
        
        # parameters from resnet paper
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=self.dim, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.dim)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=7, stride=2, padding=1)
        
        # bottleneck layers
        self.layer1 = self._build_layer(num_layers=3, out_dim=64, stride=1, padding=0) 
        self.layer2 = self._build_layer(num_layers=4, out_dim=128, stride=2, padding=1)
        self.layer3 = self._build_layer(num_layers=6, out_dim=256, stride=2, padding=1)
        self.layer4 = self._build_layer(num_layers=3, out_dim=512, stride=2, padding=1)
        
        # pool/fc
        self.adapool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(2048, self.num_classes)
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.adapool(x)
        out = self.fc(x)
        
        return out
         
    def _build_layer(self, num_layers, out_dim, stride, padding):
        '''
        layer1: (no image reduction)
            in_dim: 64
            out_dim: 64
            stride: 1
            padding: 0
        
        layer2: 56 -> 28
            in_dim: 256
            out_dim: 128
            stride: 2
            padding: 1
            
        layer3: 28 -> 14
            in_dim: 516
            out_dim: 256
            stride; 2
            padding: 1
        
        layer4: 14 -> 7
            in_dim: 1028
            out_dim: 512
            stride: 2
            padding: 1
            
        AdaptiveAvgPool2d: 7 -> 1
        '''
        # apply stride (image shape reduction) in first bottleneck block of each layer
        layers = []
        layers.append(
            Bottleneck(in_dim=self.dim, out_dim=out_dim, expansion=self.expansion, stride=stride, padding=padding, downsample=True)
        )
        
        self.dim = self.expansion * self.dim # dimension expansion
        for i in range(1, num_layers):
            layers.append(
                Bottleneck(in_dim=self.dim, out_dim=out_dim, expansion=self.expansion, stride=1, padding=0)
            )
            
        return nn.Sequential(*layers)

In [86]:
# pytorch bottleneck architecture applies stride in conv3x3 with stride=2 and padding=1
# other architectures apply stride in 1st conv1x1 with stride=2 and padding=0

# apply stride (image shape reduction) in first bottleneck block of each layer

# bottleneck architecture: 
    # conv1x1 dimension reduction
    # conv3x3 image shape reduction
    # conv1x1 dimension expansion

class Bottleneck(nn.Module):
    def __init__(self, in_dim, out_dim, expansion, stride, padding, downsample=False):
        super().__init__()
        self.expansion = expansion
        
        # layers
        self.conv1 = nn.Conv2d(in_channels=in_dim, out_channels=out_dim, kernel_size=1, stride=1)
        self.bn1 = nn.BatchNorm2d(out_dim)
        self.conv2 = nn.Conv2d(in_channels=out_dim, out_channels=out_dim, kernel_size=3, stride=stride, padding=padding)
        self.bn2 = nn.BatchNorm2d(out_dim)
        self.conv3 = nn.Conv2d(in_channels=out_dim, out_channels=out_dim * self.expansion, kernel_size=1, stride=1)
        self.bn3 = nn.BatchNorm2d(out_dim * self.expansion)
        self.relu = nn.ReLU(inplace=True)
        
        self.downsample = nn.Identity()
        if downsample:
            self.downsample = nn.Conv2d(in_channels=in_dim, out_channels=out_dim * self.expansion, kernel_size=1, stride=1)
        
    def forward(self, x):
        out = self.relu(self.bn1(self.conv1(x)))
        out = self.relu(self.bn2(self.conv2(x)))
        out = self.bn3(self.conv3(x))
        
        identity = self.downsample(x) # identity mapping
        
        out += identity
        out = self.relu(out)
        
        return out

In [91]:
params = {'dim':64, 'expansion':4, 'num_classes':10}

In [92]:
model = ResNet(**params)

In [93]:
img = torch.rand(1, 3, 224, 224)